In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
# from mplfinance import candlestick_ohlc
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates



import numpy as np

from os.path import exists

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import seaborn as sns

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300})


In [3]:
df = pd.read_csv('pk-data-feeder/data/kucoinfutures/ALGOUSDTM-15m.csv')

df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
df

FileNotFoundError: [Errno 2] No such file or directory: 'pk-data-feeder/data/kucoinfutures/ALGOUSDTM-15m.csv'

In [24]:
df = pd.read_json(r'./freq-user-data/data/binance/ALGO_USDT-1h.json')
df.columns = ['ts', 'o', 'h', 'l', 'c', 'v']

df['date'] = pd.to_datetime(df['ts'], unit='ms', utc=False)

# df = pd.read_json(r'./freq-user-data/data/kucoin/BTC_USDT-1h.json')
# df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
# df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')


In [25]:
df['lo'] = df.o.apply(np.log)
df['lh'] = df.h.apply(np.log)
df['ll'] = df.l.apply(np.log)
df['lc'] = df.c.apply(np.log)

df['ma1'] = df.c.ewm(span=14, adjust=False).mean()
df['ma2'] = df.c.ewm(span=26, adjust=False).mean()
df['ma3'] = df.c.ewm(span=50, adjust=False).mean()

df['lma1'] = df.lc.ewm(span=14, adjust=False).mean()
df['lma2'] = df.lc.ewm(span=26, adjust=False).mean()
df['lma3'] = df.lc.ewm(span=50, adjust=False).mean()


df['fair_c_ma1'] = df.lc - df.lma1
df['fair_c_ma2'] = df.lc - df.lma2
df['fair_c_ma3'] = df.lc - df.lma3

df['fair_ma1_ma2'] = df.lma1 - df.lma2
df['fair_ma2_ma3'] = df.lma2 - df.lma3
df['fair_ma1_ma3'] = df.lma1 - df.lma3

df['lret'] = df.c.apply(np.log).diff()
df = df.tail(-50).copy()

In [100]:
# pd.Timestamp('now').floor('D') + pd.Timedelta(-7, unit='D')

# start_time = datetime(2021,5,1)

start_time = pd.Timestamp('now') + pd.Timedelta(-100, unit='W')
# start_time
ddf = df.loc[df.index >= start_time].copy()
# ddf = df.loc[df['time'] >= start_time]
# ddf
dlen = len(ddf.index)
dlen

9357

In [26]:
ddf = df.copy()
strategy = '3-emas'
strategy_params_json = f'./strategy-params/{strategy}.json'

fee:float = 0.001
# capital = 10000
# risk_per_trade_perc = 0.01
# risk = 0.01
dlen = len(ddf)

sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

strategy_params = {
    # 'mav1': {'type': 'int', 'value': 21, 'step': 1},
    # 'mav2': {'type': 'int', 'value': 50, 'step': 1},
    # 'mav3': {'type': 'int', 'value': 50, 'step': 5},
    # 'fair_ma1_ma2': {'type': 'float', 'value': 0.01, 'step': 0.0005},
    'ma2_ma3': {'type': 'float', 'value': 0.01, 'step': 0.002},
    'c_mav2': {'type': 'float', 'value': 0.01, 'step': 0.002},
    'c_mav1': {'type': 'float', 'value': 0.01, 'step': 0.002},
    'stop_gap': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    'slipage': {'type': 'float', 'value': 0.001, 'step': 0.0005},
    'risk': {'type': 'float', 'value': 0.01, 'step': 0.005},
    'sl': {'type': 'float', 'value': 0.01, 'step': 0.001},
    'r2r': {'type': 'float', 'value': 2./1., 'step': 0.05},
    'wbtp': {'type': 'float', 'value': 0.1, 'step': 0.05},
    'trail': {'type': 'float', 'value': 0.02, 'step': 0.005},
}
for pk, pv in strategy_params.items():
    pv['wdg'] = globals()[(pv['wdg'] if 'wdg' in pv else pv['type'].capitalize() + 'Text')](
        description=pk,disabled=False,continuous_update=False,
        value=pv['value'], step=pv['step']
    )
strategy_params = {"w2log": {'wdg': sl_n2}, "w": {'wdg': sl_w}, **strategy_params}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

wst = wed = wsz = buy_sig = g_buy_sig_cum = buy_cond = False
enter_sig_ind = enter_sig_arr = exit_sig_arr = False
wdf = wadf = False
tdf = pd.DataFrame(columns = ['side', 'enter_date', 'enter_price', 'buy_price', 'exit_date', 'exit_price', 'sell_price', 'qty', 'pnl', 'capital'])

def printer(w2log, w, ma2_ma3, c_mav1, c_mav2, r2r, stop_gap, slipage, sl, risk, wbtp, trail):
    # global capital
    capital = 10000
    global wst, wed, wsz, wdf, tdf
    global enter_sig_ind, enter_sig_arr, exit_sig_arr, stop_sig_arr, target_sig_arr
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))

    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = ddf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)


    wdf = wdf.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})
    wdf['body_top'] = wdf[['open', 'close']].max(axis=1)
    wdf['body_btm'] = wdf[['open', 'close']].min(axis=1)
    wdf['wick_top'] = wdf.high - wdf.body_top
    wdf['wick_btm'] = wdf.body_btm - wdf.low
    wdf['wick_top_perc'] = wdf.wick_top / wdf.open * 100
    wdf['wick_btm_perc'] = wdf.wick_btm / wdf.open * 100
    wdf['range'] = wdf.high - wdf.low
    wdf['body_range'] = wdf.body_top - wdf.body_btm
    wdf['body_range_perc'] = wdf.body_range / wdf.open * 100
    # wdf['perc_chg'] = (wdf.close - wdf.open) / wdf.open * 100
    # wdf['mav1'] = wdf.ta.ema(length=mav1, sma=False)
    # wdf['mav2'] = wdf.ta.ema(length=mav2, sma=False)
    # wdf['mav3'] = wdf.ta.ema(length=mav3, sma=False)

    # wdf['direction'] = np.where(wdf.close > wdf.open, 1, -1)

    buy_sig = (
        (wdf.wick_btm_perc >= wbtp) & 
        # (wdf.direction == -1) &
        # (wdf.range < wdf.range.shift(fill_value=0)) &
        # (wdf.low < wdf.mav2) &
        # (wdf.open <= wdf.mav2) &
        (wdf.close <= wdf.ma2) &
        (wdf.fair_c_ma1 < c_mav1) &
        # (wdf.fair_c_ma1 < c_mav1) &
        # (wdf.fair_c_ma2 < c_mav2) &
        # (wdf.close >= wdf.mav3) &
        # (wdf.low <= wdf.mav3) &
        # (wdf.perc_chg > 0) &
        # (wdf.mav1 > wdf.mav2) &
        # (wdf.mav2 > wdf.mav3) &
        # (wdf.fair_ma1_ma3 >= fair_ma2_ma3) &
        (wdf.fair_ma2_ma3 >= ma2_ma3)
        
    ).shift(1, fill_value=False)


    # sell_sig = (
    #     (wdf.high2body >= hw2b) & 
    #     (wdf.direction == 1)
        
    # ).shift(1, fill_value=False)

    # buy_sig = buy_cond & (~buy_cond).shift(1, fill_value=False)

    enter_sig_arr = np.full(wlen, False)
    exit_sig_arr = np.full(wlen, False)
    in_pos_arr = np.full(wlen, 0)
    roi_ret_arr = np.full(wlen, 0.)
    pnl_arr = np.full(wlen, 0.)
    cap_arr = np.full(wlen, 0.)

    enter_price_arr = np.full(wlen, np.nan)
    stop_price_arr = np.full(wlen, np.nan)
    target_price_arr = np.full(wlen, np.nan)
    exit_price_arr = np.full(wlen, np.nan)
    capital_arr = np.full(wlen, np.nan)

    # pos_arr = np.full(wlen, 0.)
    vopen, vclose, vhigh, vlow = wdf.open, wdf.close, wdf.high, wdf.low
    vdates = wdf.index.values
    mav1a, mav2a = wdf.ma1, wdf.ma2

    is_long = is_short = in_pos = enter_date = False
    enter_price, stop_price, target_price, buy_price, sell_price, pos_qty = 0., 0., 0., 0., 0., 0.
    for i in range(0, wlen):
        in_pos_arr[i] = in_pos
        exit_price, pnl = 0., 0.

        if not in_pos and buy_sig[i]:
            is_long, in_pos = True, True
            enter_date = vdates[i]
            enter_sig_arr[i] = True
            enter_price = vopen[i] 
            buy_price = enter_price * (1 + fee) * (1 + slipage)
            # stop_price = vlow[i-4:i-1].min() * (1. - stop_gap)
            stop_price = enter_price * (1 - sl) * (1. - stop_gap)
            target_price = enter_price + (enter_price - stop_price) * r2r
            pos_qty = capital * risk /(enter_price - stop_price)
            pos_qty = max(pos_qty, capital / 2 / enter_price)
            enter_price_arr[i] = enter_price
            target_price_arr[i] = target_price
            stop_price_arr[i] = stop_price

            if vlow[i] < stop_price:
                exit_price = stop_price

        if is_long and exit_price == 0:
            if vlow[i] <= stop_price:
                exit_price = stop_price
            elif vhigh[i] >= target_price:            
                exit_price = target_price 
            # elif vopen[i-1] < enter_price:
            #     exit_price = vopen[i]
            elif mav1a[i-1] < mav2a[i-1] and enter_price > vopen[i]:
                exit_price = vopen[i]

            elif trail != 0 and stop_price < enter_price and vopen[i] > enter_price * (1 + trail):
                stop_price = enter_price * (1 + 2 * fee)
            # elif sell_sig[i]:
            #     exit_price = vopen[i]
            # elif green_ins_green[i-1]:
            #     exit_price = vopen[i]


        if is_long and exit_price != 0: 
            sell_price = exit_price * (1 - fee) * (1 - slipage)
            exit_price_arr[i] = exit_price
            exit_sig_arr[i] = True
            pnl = (sell_price - buy_price) * pos_qty 
            pnl_arr[i] = pnl
            exit_date = vdates[i]
            capital += pnl
            # tdf = tdf.append({'side': 'long', 'enter_date': enter_date, 'enter_price': enter_price, 'buy_price': buy_price, 
            #     'exit_date': exit_date, 'exit_price': exit_price, 'sell_price': sell_price, 'qty': pos_qty, 'pnl': pnl, 'capital':capital}, ignore_index=True)

            # tdf.iloc[len[tdf]] = ['long', enter_date, enter_price, buy_price, exit_date, exit_price, sell_price, pos_qty, pnl, capital]
            # roi_ret_arr[i] = ret / enter_price - 2 * fee
            in_pos, is_long = False, False

    wdf['pnl'] = pnl_arr
    wdf['cum_pnl'] = wdf.pnl.cumsum()
    # wdf['cap'] = capital
    # wdf['in_pos'] = in_pos_arr
    wdf['enter_sig'] = enter_sig_arr
    wdf['exit_sig'] = exit_sig_arr
    wdf['enter_price'] = np.where(enter_sig_arr, enter_price_arr, np.nan)
    wdf['target_price'] = np.where(enter_sig_arr, target_price_arr, np.nan)
    wdf['stop_price'] = np.where(enter_sig_arr, stop_price_arr, np.nan)
    wdf['exit_price'] = np.where(exit_sig_arr, exit_price_arr, np.nan)

    enter_sig_ind = wdf[wdf.enter_sig].index
    exit_sig_ind = wdf[wdf.exit_sig].index
    
    # positions_start_row = enter_groups.head(1)
    # positions_end_row = exit_groups.head(1)
    # positions_min_poscumlret = exit_groups.poscumlret.min()
    # positions_max_poscumlret = exit_groups.poscumlret.max()

    # wdf['poscumplret'] = wdf.totcumlret * wdf.in_pos.astype(int)

    # wexits = wdf.loc[exit_sig_ind]
    # wprofits = wexits[wexits.pnl > 0]
    # wlosses = wexits[wexits.pnl <= 0]

    
#     with open(f'./strategy-results/{datetime.now():strat_%Y-%m-%d}.log', "a") as f: 
#         f.write(json.dumps(dict({   
#             "dt": f'{datetime.now():%Y-%m-%d %HH:%MM:%SS}',
#             "period_start": f'{ddf.index[0]}',
#             "period_end": f'{ddf.index[-1]}',
#             "params": dict({ 'sl': sl, 'ts': ts, 'tp': tp, 'ma1n': ma1n, 'ma2n': ma2n, 'ma1c': ma1c, 'ma2c': ma2c, 'trn': trn, 'mrn': mrn, 'xrr': xrr
# #                         'std': std, 'stk': stk, 'stsm': stsm, 'stt': slStT.value
#                            }),
#             "strategy": "2EMASTH",
#             "profit": floor(100* (trades.iloc[-1].cap / capital - 1)) if len(trades.index) else 0. 
#         })
#     )); f.write("\n");
    
    
    plt.close("all")
    
    with plt.rc_context():
        kwargs = dict(warn_too_much_data=100000)

        fig = plt.figure(constrained_layout=True,figsize=(16,5))
        gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

        gs00 = gs0[0,0].subgridspec(3, 1, height_ratios=[3,1,1])
        gs01 = gs0[0,1].subgridspec(4, 1)

        ax001 = fig.add_subplot(gs00[0,0])
        ax002 = fig.add_subplot(gs00[1,0], sharex=ax001)
        ax003 = fig.add_subplot(gs00[2,0], sharex=ax001)

        ax101 = fig.add_subplot(gs01[0,0])
        ax102 = fig.add_subplot(gs01[1,0])
        ax103 = fig.add_subplot(gs01[2,0])
        
        locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
        formatter = mdates.ConciseDateFormatter(locator)
        ax001.xaxis.set_major_locator(locator)
        ax001.xaxis.set_major_formatter(formatter)

        mpf.plot(wdf,
            **{**kwargs, **dict(volume=False, style='binance',type='candle')},  #figsize=(15, 5)
            show_nontrading=True,
            # vlines=dict(vlines=vlines, linewidths=(1,0.5), colors=('g','c'), linestyle='-.'),
            ax=ax001,
            # vlines=['2021-03-15 04:00:00'],
            # addplot=ap2
            )
        ax001.plot(wdf.ma1, color='orange')
        ax001.plot(wdf.ma2, color='green')
        ax001.plot(wdf.ma3, color='blue')
        
        max_pnl = wdf[['cum_pnl']].max().max()
        min_pnl = wdf[['cum_pnl']].min().min()
        ax002.plot(wdf.cum_pnl, lw=1, color='r')
        # ax002.vlines(wdf[wdf.g_cond_beg_end].index, min_pnl, max_pnl, lw=0.5, color="c")

        # ax003.plot(wdf.fair_c_ma2, lw=1, color='g')
       
    # with plt.rc_context():
    #     kwargs = dict(warn_too_much_data=100000)
    #     # fig = mpf.figure(figsize=(15,6))
    #     # ax1 = fig.add_subplot(2,1,1)
    #     # ax11 = ax1.twinx()

    #     ap2 = [ 
    #             mpf.make_addplot(wdf.cum_pnl,color='g',panel=1),
    #             # mpf.make_addplot(tdf.capital,color='g',panel=1),
    #             # wdf[wdf.enter_price] ,
    #             *(
    #                 [mpf.make_addplot(wdf.enter_price,type='scatter',markersize=10,marker='^',color='blue', panel=0)] 
    #                     if len(wdf[~wdf.enter_price.isnull()]) else []
    #             ),
    #             *([mpf.make_addplot(wdf.exit_price,type='scatter',markersize=10,marker='v', color='blue',panel=0)]
    #                     if len(wdf[~wdf.exit_price.isnull()]) else []
    #             ),
    #             mpf.make_addplot(wdf.mav2,type='line',panel=0),
    #             mpf.make_addplot(wdf.mav1,type='line', panel=0),
    #             mpf.make_addplot(wdf.mav3,type='line', panel=0),
    #             # mpf.make_addplot(wdf.cum_pnl,color='b',panel=1)
    #             # mpf.make_addplot(df['LowerB'],color='b',panel=1),
    #         ]
    #     mpf.plot(wdf,**{**kwargs, **dict(volume=False, style='binance',type='candle', figsize=(15, 6))},addplot=ap2)

        # mpf.plot(hdf,ax=ax1,**kwargs, style='binance')

        wclose_min = wdf.close.min()
        wclose_max = wdf.close.max()
       
    plt.show()
    # print(pf.stats())
    # print(pf.returns_stats())
    
if exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];
                                        

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()})
display(ui, out)

Output()

In [22]:
1 if len(wdf[~wdf.enter_price.isnull()]) else 0

0

In [11]:
wdf

,ts,open,high,low,close,v,date,lo,lh,ll,...,lma1,lma2,lma3,fair_c_ma1,fair_c_ma2,fair_c_ma3,fair_ma1_ma2,fair_ma2_ma3,fair_ma1_ma3,lret
date,,,,,,,,,,,,,,,,,,,,,
2021-01-13 18:00:00,1610560800000,0.4445,0.4464,0.4387,0.4389,1119329.66,2021-01-13 18:00:00,-0.810805,-0.806540,-0.823939,...,-0.853534,-0.869071,-0.870999,0.030050,0.045587,0.047515,0.015537,0.001927,0.017465,-0.012903
2021-01-13 19:00:00,1610564400000,0.4392,0.4415,0.4379,0.4383,1010399.11,2021-01-13 19:00:00,-0.822800,-0.817577,-0.825765,...,-0.849709,-0.865796,-0.869189,0.024858,0.040944,0.044337,0.016086,0.003393,0.019479,-0.001368
2021-01-13 20:00:00,1610568000000,0.4384,0.4467,0.4356,0.4390,1566088.59,2021-01-13 20:00:00,-0.824624,-0.805868,-0.831031,...,-0.846182,-0.862644,-0.867388,0.022926,0.039389,0.044132,0.016462,0.004743,0.021205,0.001596
2021-01-13 21:00:00,1610571600000,0.4391,0.4437,0.4380,0.4419,971017.87,2021-01-13 21:00:00,-0.823028,-0.812607,-0.825536,...,-0.842248,-0.859239,-0.865399,0.025576,0.042567,0.048727,0.016991,0.006160,0.023151,0.006584
2021-01-13 22:00:00,1610575200000,0.4415,0.4451,0.4400,0.4428,1270617.60,2021-01-13 22:00:00,-0.817577,-0.809456,-0.820981,...,-0.838566,-0.855935,-0.863408,0.023929,0.041298,0.048771,0.017369,0.007473,0.024842,0.002035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-03 21:00:00,1612386000000,0.7387,0.7397,0.7230,0.7262,1839586.69,2021-02-03 21:00:00,-0.302863,-0.301511,-0.324346,...,-0.353834,-0.375941,-0.398814,0.033904,0.056012,0.078884,0.022107,0.022872,0.044980,-0.017472
2021-02-03 22:00:00,1612389600000,0.7262,0.7435,0.7237,0.7435,1369091.82,2021-02-03 22:00:00,-0.319930,-0.296387,-0.323378,...,-0.346174,-0.370048,-0.394797,0.049788,0.073662,0.098410,0.023874,0.024749,0.048623,0.023543
2021-02-03 23:00:00,1612393200000,0.7435,0.7519,0.7342,0.7454,3262195.49,2021-02-03 23:00:00,-0.296387,-0.285152,-0.308974,...,-0.339196,-0.364403,-0.390838,0.045361,0.070569,0.097003,0.025207,0.026435,0.051642,0.002552
